<a href="https://colab.research.google.com/github/Ketian-Wang/RobotLearning/blob/main/RobotLearning_proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup


In [ ]:
!git clone https://github.com/roamlab/mecs6616_sp23_project3.git
!cp -av /content/mecs6616_sp23_project3/* /content/
!pip install ray

Cloning into 'mecs6616_sp23_project3'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 27 (delta 11), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (27/27), 23.05 KiB | 1.65 MiB/s, done.


# Starter Code Explanation

This project uses two 3-link arms, one called arm_teacher (blue) and the other called arm_student (red), as shown in the image below. For each test, a constant torque will be applied to the first joint of both arms for 5 seconds. arm_teacher is moving according to the provided ground truth forward dynamics and your job is to use deep learning to train the arm_student to learn the forward dynamics of the arm_teacher so that it can imitate its behavior. The forward dynamics is a function that takes in the current state of and an action applied to the arm, and then computes the new state of the arm. This project uses a time step of 0.01 second, meaning each time we advance the simulation, we compute the forward dynamics for 0.01 second. In the example image, the student arm is not updating its state and remains static but we will make it move after training is done.



<div>
<img src="https://github.com/roamlab/mecs6616_sp23_project3/blob/master/imgs/example.png?raw=true" width="600"/>
</div>

The interface for controlling the robot is defined in the `Robot` class in `robot.py` file. Each robot is initialized with a corresponding forward dynamics (the base class for forward dynamics definition is in `arm_dynamic_base.py`). The arm_teacher is initialized with the provided ground truth forward dynamics, as defined in `arm_dynamics_teacher.py`. You are welcome to look in-depth into this file to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - this is recommended but not necessary to successfully complete this assignment. The state of each arm is defined with a (6,1)-dimensional numpy array (three joint positions in radians + three joint velocities in radians per second). An action is defined as the three toques (in Nm) applied to the three joints respectively, which is a (3,1) numpy array. **Throughout this project, the problem is simpllified by only applying a torque to the first joint, so the actions always look like `[torque,0,0]`.** Also, when scoring your model the robot will always start off in a hanging position, meaning an initial state of `[-pi/2,0,0,0,0,0]` so if the collected data from part 1 looks similar, the model will perform better. The `robot.py` file provides you with some functions to set/get the state and set the action for the arm.

`geometry.py` provides some geometry functions and `render.py` defines how the visualization is rendered. These two files are not of particular interest for completing this project.

# Part I. Data collection.

Firstly, the project need to collect a dataset for training the forward dynamics. After running the cell, it will generate a pickle file `data.pkl` that contains a data dictionary `data = {'X': X, 'Y': Y}`. The shape of `data['X']` should be (`num_samples`, 9), the first 6 elements are state and the last 3 elements are the action. The shape of `data['Y']` should be (`num_samples`, 6), which saves the next state after applying the action using the ground truth forward dynamics of arm_teacher. 

**After the data file is generated, `data.pkl` will appear under the 'Files' icon in the left sidebar.** 





In [ ]:
import numpy as np
import os
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
import pickle
import math
from render import Renderer
import time
np.random.seed(0)

# DO NOT CHANGE
# Teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# ---
# You code starts here. X and Y should eventually be populated with your collected data

num_state = 1
num_action = 400
num_steps = 1000

X = np.zeros((num_state * num_action * num_steps, arm_teacher.dynamics.get_state_dim() + arm_teacher.dynamics.get_action_dim()))
Y = np.zeros((num_state * num_action * num_steps, arm_teacher.dynamics.get_state_dim()))

# GUI visualization, this will drastically reudce the speed of the simulator!
gui = False

# Initialize the GUI
if gui:
    renderer = Renderer()
    time.sleep(1)

from tqdm import tqdm
for count_state in tqdm(range(num_state)):
  # random initial states. 
  initial_state = np.zeros((arm_teacher.dynamics.get_state_dim(), 1))  # position + velocity
  initial_state[0] = -math.pi / 2.0
  print("been here", count_state)
  for count_action in range(num_action):
    # random action. Also let torque of joints 2&3 to be none zero 
    arm_teacher.set_state(initial_state)
    action = np.append((-2 + count_action * 4 / num_action), np.zeros((2,1))).reshape(-1,1)
    arm_teacher.set_action(action)

    for count_step in range(num_steps):
      # Get the current state
      state = arm_teacher.get_state()
      position = count_state * (num_action * num_steps) + count_action * num_steps + count_step

      X[position] = np.append(state, action)

      # The advance function will simulate the action for 1 time step
      arm_teacher.advance()
      if gui:
         renderer.plot([(arm_teacher, 'tab:blue')])
    
      # Get the new state after advancing one time step
      new_state = arm_teacher.get_state()
      new_state = new_state.reshape(1, -1)
      Y[position] = new_state
# ---
print(X.shape)
# DO NOT CHANGE
# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open( "data.pkl", "wb" ) )

  0%|          | 0/1 [00:00<?, ?it/s]

been here 0


100%|██████████| 1/1 [04:36<00:00, 276.88s/it]


(400000, 9)


# Part 2. Learning the forward dynamics.

## Training

After the data is collected, the model then need to use the collected dataset to learn the forward dynamics. 

The model creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split. This cell should save the model that it trains. 

In machine learning, it is a very good practice to save not only the final model but also the checkpoints, such that you have a wider range of models to choose from. 


### Important: choosing the best model

The code will keep track of the checkpoint with the smallest loss on the test set. The path of that checkpoint will be saved to the variable `model_path`. In the evaluation code, the checkpoint from `model_path` will be loaded and evaluated.  

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import argparse
import time
import pickle
np.set_printoptions(suppress=True)

torch.manual_seed(0)

class DynamicDataset(Dataset):
    def __init__(self, data_file):
        data = pickle.load(open(data_file, "rb" ))
        # X: (N, 9), Y: (N, 6)
        self.X = data['X'].astype(np.float32)
        self.Y = data['Y'].astype(np.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


class Net(nn.Module):
    # ---
    # Your code goes here
    def __init__(self):
      super(Net, self).__init__()
      self.layer1 = nn.Linear(9, 128)
      self.layer2 = nn.Linear(128,32)
      self.layer3 = nn.Linear(32,3) # output three accelartions

    def forward(self, x):
      old_x = x
      y = torch.nn.functional.relu(self.layer1(x))
      y = torch.nn.functional.relu(self.layer2(y))    
      y = self.layer3(y)
      y = acce2PosAndSpeed(old_x, y)

      return y


    def predict(self, features):
      self.eval()
      features = torch.from_numpy(features).float()
      features = features.unsqueeze(0)
      return self.forward(features).detach().numpy()

    # ---

network = Net()
learning_rate = 0.0001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(network.parameters(),lr = learning_rate) #, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
batch_size = 100

def train(model, loader):
    model.train()
    total_loss = 0.0
    for i, data in enumerate(loader):
      x = data[0]
      y = data[1]
      optimizer.zero_grad()
      predictions = model.forward(x)
      loss = criterion(predictions, y)
      loss.backward()
      total_loss += loss.item()
      optimizer.step()
    scheduler.step()
    return total_loss/(i+1)



def test(model, loader):
    model.eval()

    # --
    # Your code goes here
    test_loss = 0.0
    for i, data in enumerate(loader):
      x = data[0]
      y = data[1]
      predictions = model.forward(x)
      loss = criterion(predictions, y)
      loss.backward()
      test_loss += loss.item()
    return test_loss/(i+1)

def acce2PosAndSpeed(x, acce):
    # print(x.shape)
    pos = x[:, :3]
    vel = x[:, 3:6]

    new_vel = vel + acce * 0.01
    new_pos = pos + vel * 0.01 + acce * 0.5 * 0.01 * 0.01

    output = torch.cat([new_pos, new_vel], dim=1)
    return output    



# The ratio of the dataset used for testing
split = 0.2

# We are only using CPU, and GPU is not allowed.
device = torch.device("cpu")

dataset = DynamicDataset('data.pkl')
dataset_size = len(dataset)
test_size = int(np.floor(split * dataset_size))
train_size = dataset_size - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True, batch_size = batch_size)

# The name of the directory to save all the checkpoints
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
model_dir = os.path.join('models', timestr) 

# Keep track of the checkpoint with the smallest test loss and save in model_path
model_path = None



epochs = 50
min_loss = 100
from tqdm import tqdm
for epoch in tqdm(range(1, 1 + epochs)):
    # ---
    # Your code goes here
    total_loss = train(network, train_loader)
    print("train loss:", total_loss)

    test_loss =  test(network, test_loader)
    print("test loss:", test_loss)
    
    if test_loss < min_loss:
      min_loss  = test_loss
      model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
      if not os.path.exists(os.path.join(model_dir, model_folder_name)):
          os.makedirs(os.path.join(model_dir, model_folder_name))
      torch.save(network.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
      model_path = os.path.join(model_dir, model_folder_name, 'dynamics.pth')


320000


  0%|          | 0/50 [00:00<?, ?it/s]

train loss: 0.000205328787249357


  2%|▏         | 1/50 [00:12<09:52, 12.08s/it]

test loss: 0.00018242800673874628
train loss: 0.00013910224935173687


...

test loss: 2.0388287706119e-07
train loss: 1.9951052377287226e-07


 98%|█████████▊| 49/50 [07:15<00:08,  8.50s/it]

test loss: 2.006867245984978e-07
train loss: 1.9668866521715955e-07


100%|██████████| 50/50 [07:24<00:00,  8.90s/it]

test loss: 1.932903591139734e-07


## Prediction

After done with training, the saved checkpoint (in function init_model) is loaded and then useed to predict the new state given the current state and action (in function dynamics_step).

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # Initialize the model loading the saved model from provided model_path
        
        self.model = Net()
        self.model.load_state_dict(torch.load(model_path))
        self.model.eval()
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # Use the loaded model to predict new state given the current state and action
            # Output should be an array of shape (6,1)
            x = np.append(state, action)
            new_state = self.model.predict(x)
            return new_state.T
            
            # ---
        else:
            return state

# Evaluation and Grading

The total number of points for this project is 15. There are 3 types of tests, each is worth 5 points. 

**For each type, there are 50 tests.** For each test, a score of 1, 0.5, or 0 is generated. The final grade for each type is the averaged score across 50 tests * 5. 

- *Type 1*: for each test, a constant torque randomly sampled from [-1.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If the MSE (Mean Squred Error) between the predicted arm state (arm_student) and the ground truth arm state (arm_teacher) is < 0.0005, score 1 will be assigned for this test. If 0.0005 <= MSE < 0.008, score 0.5 will be assigned for this test. Otherwise you get 0.
- *Type 2*: for each test, a torque that linearly increases from 0 to a random torque in [0.5Nm, 1.5Nm] is applied to the first joint of the arm for 5 seconds. If MSE < 0.0005, score 1 will be assigned for this test. If 0.0005 <= MSE < 0.008, score 0.5 will be assigned for this test. Otherwise you get 0. 
- *Type 3*: for each test, one torque is applied for the first 2.5 seconds and another torque is applied for the remaining 2.5 seconds. Both torques are sampled from [-1Nm, 1Nm]. If MSE < 0.015, score 1 will be assigned for this test. If 0.015 <= MSE < 0.05, score 0.5 will be assigned for this test. Otherwise you get 0. 


In [ ]:
# DO NOT CHANGE
# Set up grading

# Make sure model_path is correctly set
print(model_path)
import importlib
import score
importlib.reload(score)

# Create the teacher arm
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
arm_teacher = Robot(dynamics_teacher)

# Create the student arm
dynamics_student = ArmDynamicsStudent(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01
)
if model_path is not None:
  dynamics_student.init_model(model_path, num_links=3, time_step=0.01, device=torch.device('cpu'))
arm_student = Robot(dynamics_student)

models/2023-04-01_02-08-43/epoch_0050_loss_0.00000019/dynamics.pth


In [ ]:
# Test on randomly sampled torques from [-1.5, 1.5]
score.score_random_torque(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque = 0.8139619298002381 Nm)

average mse: 6.323112087717268e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque = -1.4377441519217955 Nm)

average mse: 0.0004937247228007773
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque = 0.4009447047788264 Nm)

average mse: 3.0213453034725436e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque = 0.7464116476158358 Nm)

average mse: 3.4612379813069965e-05
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque = -0.004478963092228838 Nm)

average mse: 2.038855146621431e-07
Score: 1/1
----------------------------------------


----------------------------------------
TEST 6 (Torque = -0.825610063407457 Nm)

average mse: 6.451971877853717e-05
Score: 1/1
----------

KeyboardInterrupt: ignored

In [ ]:
# Test on torques that linearly increase from 0 to a random number from [0.5, 1.5]
score.score_linear_torques(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque 0 -> 1.2713206432667459 Nm)

average mse: 0.00020128817059078324
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque 0 -> 0.5207519493594015 Nm)

average mse: 7.6388676327018e-06
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque 0 -> 1.1336482349262753 Nm)

average mse: 0.00015544526011794814
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque 0 -> 1.2488038825386119 Nm)

average mse: 0.00019504395199310836
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque 0 -> 0.9985070123025904 Nm)

average mse: 0.00010885304675862717
Score: 1/1
----------------------------------------


----------------------------------------
TEST 6 (Torque 0 -> 0.7247966455308477 Nm)

average mse: 1.823558660216139e-05
Score: 

In [ ]:
# Test on one torque applied to the first 2.5s and another torque applied to the second 2.5s
# Both torques are sampled from [-1, 1]
score.score_two_torques(arm_teacher, arm_student, gui=False)


----------------------------------------
TEST 1 (Torque 1 = 0.542641286533492 Nm,  Torque 2 = -0.21494151210682544 Nm)

average mse: 0.0007055059512260844
Score: 1/1
----------------------------------------


----------------------------------------
TEST 2 (Torque 1 = -0.958496101281197 Nm,  Torque 2 = -0.8130792508826994 Nm)

average mse: 0.0001592978891771538
Score: 1/1
----------------------------------------


----------------------------------------
TEST 3 (Torque 1 = 0.26729646985255084 Nm,  Torque 2 = 0.6422113156738569 Nm)

average mse: 0.0001346399035160972
Score: 1/1
----------------------------------------


----------------------------------------
TEST 4 (Torque 1 = 0.4976077650772237 Nm,  Torque 2 = -0.6976959607148723 Nm)

average mse: 0.0013735120058505654
Score: 1/1
----------------------------------------


----------------------------------------
TEST 5 (Torque 1 = -0.0029859753948191514 Nm,  Torque 2 = -0.23177110261560085 Nm)

average mse: 1.3467292798124317e-05
Sc